In [4]:
from flask import Flask, request,jsonify, Response

import joblib;
from sklearn.decomposition import PCA;
from sklearn.preprocessing import StandardScaler;
import tensorflow as tf;
import numpy as np;
import pandas as pd;
import io;
import csv;

app = Flask(__name__)

# 保存された正規化モデルを読み込む
scaler = joblib.load('scaler.pkl')

# 保存されたPCAモデルを読み込む
pca_model = joblib.load('pca_model.pkl')

# モデルを読み込む
model = tf.keras.models.load_model('model')

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['file']
    # ファイルを処理するコードをここに追加する
    if not file:
        return jsonify({'error': 'ファイルが存在しません'})

    # CSVデータを読み込む
    csv_data = io.StringIO(file.stream.read().decode("utf-8"))
    df = pd.read_csv(csv_data, sep='\t', encoding='utf-16')
    df = df.applymap(lambda x: str(x).replace(',', ''))  # カンマを削除

    df=df.drop(["Mark","Company name"],axis=1)

    
    df=df.replace("n.a.",np.float32(0.0))
    df=df.replace(",","")

    df=df.astype("float32")


    
    # データを正規化する
    standalized_df = pd.DataFrame(scaler.transform(df),columns=df.columns)

    

    # データをPCAモデルに適用して、次元を削減する
    pca_data = pca_model.transform(standalized_df)
    pca_df = pd.DataFrame(pca_data)


    # モデルを使用して予測を行う
    predictions = model.predict(pca_df.values)

    # 予測値をCSV形式のテキストに変換する
    output = io.StringIO()
    writer = csv.writer(output)
    writer.writerow(['Prediction'])
    for prediction in predictions:
        writer.writerow([prediction])

    # CSVをJSON形式に変換する
    response = Response(
        output.getvalue(),
        mimetype='text/csv',
        headers={"Content-disposition":
                 "attachment; filename=predictions.csv"})

    return response


if __name__ == '__main__':
    app.run()

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)